In [1]:
import pickle
import numpy as np
import pandas as pd

import torch
from torch_geometric.data import Data

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
design_matrix = pd.read_csv("aki_data/design_matrix.tsv", sep="\t")

In [5]:
def load_graph(patient_id, type):
    if type == 1:
        with open(f'graph_data/type1/type1_patient_graph_{patient_id}.pkl', 'rb') as f:
            graph = pickle.load(f)
    elif type == 2:
        with open(f'graph_data/type2/type2_patient_graph_{patient_id}.pkl', 'rb') as f:
            graph = pickle.load(f)
    return graph

def map_edges_to_indices(edge_list):
    # Create a unique mapping for each node
    unique_nodes = set([node for edge in edge_list for node in edge])
    node_to_idx = {node: idx for idx, node in enumerate(unique_nodes)}

    # Map edges to indices
    edge_index = [[node_to_idx[edge[0]], node_to_idx[edge[1]]] for edge in edge_list]
    return torch.tensor(edge_index, dtype=torch.long).t().contiguous()


In [16]:
train_graph_data= []
test_graph_data = []
patient_ids = design_matrix['sample'].values

for patient_id in patient_ids:
    graph = load_graph(patient_id, type=2)  # Load the graph as before

    # Convert NetworkX graph to PyTorch Geometric Data
    edge_index = map_edges_to_indices(graph.edges)

    x = torch.tensor([graph.nodes[node].get('abundance', 0) for node in graph.nodes], dtype=torch.float)
    y = torch.tensor([design_matrix[design_matrix['sample'] == patient_id]['group'].iloc[0]], dtype=torch.long)
    graph_data = Data(x=x, edge_index=edge_index, y=y)

    if 'M2012' in patient_id:
        train_graph_data.append(graph_data)
    else:
        test_graph_data.append(graph_data)

True